IMPORTS

In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from datasets import tgs_sic
from nets import inception_resnet_v2
from nets import nets_factory
from preprocessing import inception_preprocessing

slim = tf.contrib.slim

c:\users\johnny\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


CONFIGURE VARIABLES

In [2]:
model_name = "inception_v3"
dataset_name = "tgs-sic"
dataset_split_name = "train"
dataset_dir = "tmp/dataset"

batch_size = 8
max_number_of_steps = 2000
train_dir = "./tmp/tgs-sic-models/test-model/"

DEFINE FUNCTIONS

In [3]:
def load_batch(dataset, batch_size=8, height=299, width=299, is_training=True):
    data_provider = slim.dataset_data_provider.DatasetDataProvider(dataset)

    image, label = data_provider.get(['image', 'label'])

    image = inception_preprocessing.preprocess_image(
        image,
        height,
        width,
        is_training)

    images, labels = tf.train.batch(
        [image, label],
        batch_size=batch_size,
        allow_smaller_final_batch=True)

    return images, labels


MAIN

In [4]:
dataset = tgs_sic.get_split('train', dataset_dir)

# load batch of dataset
images, labels = load_batch(
    dataset,
    batch_size,
    is_training=True)

# run the image through the model
cnn1 = nets_factory.get_network_fn(
    model_name,
    num_classes=2,
    weight_decay=0.00004,
    is_training=True)
#cnn2 = nets_factory.get_network_fn(
#    model2,
#    num_classes=11,
#    weight_decay=0.00004,
#    is_training=True)
logits = cnn1(images)
#net2, end_points2 = cnn2(images)
#for endpoint in end_points1:
#    print(endpoint)
    
#for endpoint in endpoints2:
#    print(endpoint)

#tf.shape(net1)
#tf.shape(net2)
#net = tf.concat([net1, net2], 1)
#net = slim.flatten(net1)
#net = slim.dropout(net, 0.8, is_training=True, scope='Dropout')
#logits = slim.fully_connected(net, 2, activation_fn=None, scope='Logits')




# get the cross-entropy loss
one_hot_labels = slim.one_hot_encoding(
    labels,
    dataset.num_classes)
slim.losses.softmax_cross_entropy(
    logits,
    one_hot_labels)
total_loss = tf.losses.get_total_loss()
tf.summary.scalar('loss', total_loss)

# use RMSProp to optimize
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9)

# create train op
train_op = slim.learning.create_train_op(
    total_loss,
    optimizer,
    summarize_gradients=True)

# run training
slim.learning.train(
    train_op,
    logdir=train_dir,
    number_of_steps=max_number_of_steps,
    log_every_n_steps=10,
    save_summaries_secs=20)

Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.


AttributeError: 'tuple' object has no attribute 'get_shape'